In [233]:
API_KEY = 'V0JD5HR9HTVMHM2M'
SCRIPT = 'AAPL'
MAVG_PERIOD = 5

In [234]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import pandas as pd

In [235]:
class ScriptData:
    def __init__(self):
        pass
    def fetch_intraday_data(self,script_name):
        ts = TimeSeries(key=API_KEY)
        # Get json object with the intraday data and another with  the call's metadata
        data, meta_data = ts.get_intraday(script_name)
        self.data = data
    def convert_intraday_data(self,script_name):
        Data = pd.DataFrame(pd.DataFrame(self.data).T.iloc[::-1])
        Data.reset_index(inplace=True)
        return(Data)

In [236]:
#test ScriptData oN GOOGLE
script_data = ScriptData()
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

,index,1. open,2. high,3. low,4. close,5. volume
0,2022-06-16 14:30:00,2111.4500,2112.2100,2108.4100,2109.2100,31446
1,2022-06-16 14:45:00,2109.1900,2118.8900,2108.2900,2115.6900,156772
2,2022-06-16 15:00:00,2113.7900,2115.6100,2103.6578,2113.2700,172175
3,2022-06-16 15:15:00,2113.7400,2126.4800,2113.6900,2119.2100,66203
4,2022-06-16 15:30:00,2114.8550,2127.1000,2114.4200,2123.4600,63379
...,...,...,...,...,...,...
95,2022-06-21 17:15:00,2230.8800,2230.8800,2230.8800,2230.8800,3580
96,2022-06-21 17:30:00,2230.0193,2230.0193,2230.0193,2230.0193,152
97,2022-06-21 18:15:00,2228.7600,2228.7600,2225.0000,2225.0000,734
98,2022-06-21 19:15:00,2226.0000,2226.0000,2226.0000,2226.0000,104


In [237]:
def indicator1(daily_data, timeperiod):
    daily_data['SMA30'] = daily_data['4. close'].rolling(timeperiod).mean()
    new_df = pd.DataFrame(columns=['timestamp','indicator'])
    new_df['timestamp'] = daily_data['index']
    new_df['indicator'] = daily_data['SMA30']
    return(new_df)

In [238]:
# Test Indicator ON GOOGLE
indicator1(script_data.convert_intraday_data('GOOGL'),5)

,timestamp,indicator
0,2022-06-16 14:30:00,NaN
1,2022-06-16 14:45:00,NaN
2,2022-06-16 15:00:00,NaN
3,2022-06-16 15:15:00,NaN
4,2022-06-16 15:30:00,2116.16800
...,...,...
95,2022-06-21 17:15:00,2232.00800
96,2022-06-21 17:30:00,2230.68186
97,2022-06-21 18:15:00,2229.53186
98,2022-06-21 19:15:00,2228.55586


In [239]:
class Strategy:
    def __init__(self,script_name):
        self.script_name = script_name
    def get_script_data(self):
        script_data = ScriptData()
        script_data.fetch_intraday_data(self.script_name)
        self.temp_data = script_data.convert_intraday_data(self.script_name)
    def get_signals(self):
        new_df = indicator1(self.temp_data,MAVG_PERIOD) 
        new_df['close_data'] = self.temp_data['4. close']
        new_df = new_df.fillna(0)
        new_df['close_data'] = new_df['close_data'].astype(float)
        new_df['signal'] = new_df['close_data'] - new_df['indicator']

        ref_df1 = pd.DataFrame(columns={'timestamp','signal'})
        for index, row in new_df.tail(len(new_df)-MAVG_PERIOD+1).iterrows():
            if index>4:
                if (row['signal']>0 and previous_entry<0 ):
                    ref_df1 = pd.concat([ref_df1,pd.DataFrame({'timestamp' : row['timestamp'], 'signal' : 'BUY'}, index=[0])], ignore_index=True)
                if (row['signal']<0 and previous_entry>0 ):
                    ref_df1 = pd.concat([ref_df1,pd.DataFrame({'timestamp' : row['timestamp'], 'signal' : 'SELL'}, index=[0])], ignore_index=True)
            previous_entry = row['signal']
        return(ref_df1[ref_df1.columns[::-1]])  

In [240]:
strategy = Strategy(SCRIPT)
strategy.get_script_data()
strategy.get_signals()

,timestamp,signal
0,2022-06-17 12:30:00,SELL
1,2022-06-17 12:45:00,BUY
2,2022-06-17 13:45:00,SELL
3,2022-06-17 15:30:00,BUY
4,2022-06-17 16:00:00,SELL
5,2022-06-17 18:00:00,BUY
6,2022-06-17 19:15:00,SELL
7,2022-06-17 20:00:00,BUY
8,2022-06-21 05:15:00,SELL
9,2022-06-21 06:30:00,BUY
